In [23]:
import pandas as pd
import json
df = pd.read_csv("testcase-data\data\combined\all_combined_testcase_data023.csv")

In [24]:
test_case = df[['Prompt', 'Description', 'Testcase name']].to_dict()

In [25]:
from openai import OpenAI
from textwrap import dedent

client = OpenAI(api_key='')

def generate_test_case(context):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Robot Framework Test case generator expect, your task is to regenerate new set of User prompts, Description, and Testcase names that don't exist in the below **Existing Test case** ."},
            {"role": "user", "content": dedent(f"""
                Based on the Prompt, Description, and Testcase name, generate a new set of example that reflect the other possibilities, not just PayNow.

                Existing Test cases:
                ```{context}```
                                
                Ensure your response is returned in a JSON FORMAT with the following keys: Prompt, Description, and Testcase name.
                Only generate one new set of example in a JSON format and don't add ```json or this ``` to the beginning or end. just give me keys and values.
            """)}
        ],
        temperature=1,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0,
        stop=["\n\n"]
    )
    
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [8]:
# from collections import defaultdict

# # Create a defaultdict with list as the default factory
# my_dict = defaultdict(list)

# for i in range(400):
#     response = generate_test_case(test_case)
#     print("RESPONSE : \n", response)
#     print("\n\n\n")
#     my_dict['test_cases'].append(response)
    
# # Convert defaultdict to a regular dict before saving to JSON
# final_dict = dict(my_dict)

In [ ]:
import json
# Save to a JSON file
with open('test_cases.json', 'w') as json_file:
    json.dump(final_dict, json_file, indent=2)

# Print the result to verify
print(json.dumps(final_dict, indent=2))

In [3]:
def generate_response(prompt, test_case):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a Robot Framework Test case generator expert, You generate accurate response for Test cases."},
            {"role": "user", "content": dedent(f"""                
                
                EXAMPLE TEXT CASES WITH COLUMN NAMES AS PROMPT, RESPONSE, Testcase name, Description:\n\n
                
                `````````````````````````````````````````````````````````````````````````````````````````````
                
                {test_case}
                
                `````````````````````````````````````````````````````````````````````````````````````````````````                

                
                `User prompt:`
                ```{prompt}```
                
                Generate a  Robot Framework Test case **Response** for User prompt, learn from the above example to unserstand what a good response looks like.
                JUST GIVE ME THE **FULL RESPONSE** FORMATTED RIGHTLY and ensure your capture all the *** Settings ***, *** Variables *** and *** Test Cases*** for the response.
                
                ONLY RESPONSE PLEASE, DON'T GIVE ME DESCRIPTION, Testcase name or add ``` signs.
                
                
            """)}
        ],
        temperature=1,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0,
    )
    
    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [17]:
import json

# Function to read and print the content of a JSON file
def read_json_file(file_path):
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            # print(json.dumps(data, indent=2))  # Pretty print the JSON data
            return data
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"The file {file_path} is not a valid JSON file.")

# File path to the JSON file
file_path = 'test_cases.json'

# Read and print the JSON file content
data = read_json_file(file_path)


In [18]:
data

{'test_cases': ['{\n"Prompt": "Generate a robot framework test case for testing to Validate User CAN Reorder Items From Purchase History functionality of the BuyAgain system.",\n"Description": "This test case checks the validation for the functionality allowing users to reorder items from their purchase history in the BuyAgain system.",\n"Testcase name": "Validate User Can Reorder Items From Purchase History"\n}',
  '{\n  "Prompt": "Generate a robot framework test case for testing to Validate Customer Is Able To Add New Shipping Address During Order Checkout functionality of the QuickShipping datadriver.",\n  "Description": "This test case checks the validation for adding a new shipping address during the order checkout process functionality of the QuickShipping datadriver.",\n  "Testcase name": "Validate Add New Shipping Address During Order Checkout"\n}',
  '{\n  "Prompt": "Generate a robot framework test case for testing to Validate Customer Is Able To Change Account Password functi

In [20]:
# Function to safely convert JSON strings to dictionaries
def convert_to_dict(test_cases):
    converted_cases = []
    for test_case in test_cases:
        try:
            converted_cases.append(json.loads(test_case))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            # Optionally, append the original string or handle it as needed
    return converted_cases

# Converting each string to a dictionary
data["test_cases"] = convert_to_dict(data["test_cases"])

Error decoding JSON: Unterminated string starting at: line 4 column 20 (char 473)
Error decoding JSON: Unterminated string starting at: line 4 column 20 (char 436)
Error decoding JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Error decoding JSON: Expecting ',' delimiter: line 5 column 1 (char 547)
Error decoding JSON: Unterminated string starting at: line 4 column 20 (char 486)
Error decoding JSON: Unterminated string starting at: line 4 column 22 (char 490)


In [21]:
data

{'test_cases': [{'Prompt': 'Generate a robot framework test case for testing to Validate User CAN Reorder Items From Purchase History functionality of the BuyAgain system.',
   'Description': 'This test case checks the validation for the functionality allowing users to reorder items from their purchase history in the BuyAgain system.',
   'Testcase name': 'Validate User Can Reorder Items From Purchase History'},
  {'Prompt': 'Generate a robot framework test case for testing to Validate Customer Is Able To Add New Shipping Address During Order Checkout functionality of the QuickShipping datadriver.',
   'Description': 'This test case checks the validation for adding a new shipping address during the order checkout process functionality of the QuickShipping datadriver.',
   'Testcase name': 'Validate Add New Shipping Address During Order Checkout'},
  {'Prompt': 'Generate a robot framework test case for testing to Validate Customer Is Able To Change Account Password functionality of the 

In [29]:
from collections import defaultdict

# Create a defaultdict with list as the default factory
final_dict = defaultdict(list)

# Iterating 20 times
for idx in range(len(data['test_cases'])):
    
    try:
        # Sample 3 examples from dataframe
        sampled_df = df.sample(4)
        
        print(data['test_cases'][idx])
        prompt = data['test_cases'][idx]["Prompt"]
        # print(prompt)
        
        print(f"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''{idx}'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''")
        
        # Create test_case string
        test_prompt = ""
        for _, row in sampled_df.iterrows():
            test_prompt += f"Prompt: {row['Prompt']}\nResponse: {row['Response']}\nDescription: {row['Description']}\nTestcase name: {row['Testcase name']}\n\n"
            
        # Generate response
        response = generate_response(prompt, test_prompt)
        
        # Print the results
        # print(f"Sampled Test Case:\n{test_case}")
        print(f"Generated Response:\n{response}\n")
        # print("-------------------------------------------------------------------------------------------------------------------------")
        
        # Append to the lists in final_dict
        final_dict["Prompt"].append(data['test_cases'][idx]["Prompt"])
        final_dict["Response"].append(response)
        final_dict["Description"].append(data['test_cases'][idx]["Description"])
        final_dict["Testcase name"].append(data['test_cases'][idx]["Testcase name"])

    except (KeyError,TypeError) as e:
        print(f"KeyError encountered: {e}. Skipping this iteration.")
        continue


{'Prompt': 'Generate a robot framework test case for testing to Validate User CAN Reorder Items From Purchase History functionality of the BuyAgain system.', 'Description': 'This test case checks the validation for the functionality allowing users to reorder items from their purchase history in the BuyAgain system.', 'Testcase name': 'Validate User Can Reorder Items From Purchase History'}
''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''0'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Generated Response:
*** Settings ***
Documentation    Test cases for validating that the user can reorder items from purchase history functionality of the BuyAgain system
Library          SeleniumLibrary

*** Variables ***
${URL}                      https://example.com/buyagain
${USER_DETAILS}             user_details

*** Test Cases ***
Validate User Can Reorder Items From Purchase History
    [Documentation]    Test case for validating

In [31]:
final_dict

defaultdict(list,
            {'Prompt': ['Generate a robot framework test case for testing to Validate User CAN Reorder Items From Purchase History functionality of the BuyAgain system.',
              'Generate a robot framework test case for testing to Validate Customer Is Able To Add New Shipping Address During Order Checkout functionality of the QuickShipping datadriver.',
              'Generate a robot framework test case for testing to Validate Customer Is Able To Change Account Password functionality of the SecurePASS datadriver.',
              "Generate a robot framework test case for testing to Validate Customer Is Unable To Cancel An Order After It's Shipped functionality of the QuickOrder system.",
              'Generate a robot framework test case for testing to Validate User Can Add Items To Shopping Cart On Sale functionality of the ShopEZ application.',
              'Generate a robot framework test case for testing the ability to verify the user account lockout afte

In [33]:

import numpy as np
# Step 1: Determine the maximum length of the lists
max_length = max(len(lst) for lst in final_dict.values())

# Step 2: Adjust lengths by filling with NaN or truncating
for key, lst in final_dict.items():
    if len(lst) < max_length:
        # Fill shorter lists with NaN
        final_dict[key].extend([np.nan] * (max_length - len(lst)))
    elif len(lst) > max_length:
        # Truncate longer lists
        final_dict[key] = lst[:max_length]

# Step 3: Convert defaultdict to a regular dict (if necessary)
regular_dict = dict(final_dict)

# Step 4: Create the DataFrame
test_df = pd.DataFrame(regular_dict)

# Step 5: Drop rows with any NaN values
cleaned_df = test_df.dropna()


In [39]:
# Display the cleaned DataFrame
cleaned_df.sample(1).to_dict()

{'Prompt': {73: 'Generate a robot framework test case for testing to Validate Customer Unable To Complete Payment For An Invalid Order functionality of the EasyPay system.'},
 'Response': {73: '*** Settings ***\nDocumentation    Test cases for validating that the customer is unable to complete payment for an invalid order functionality of the EasyPay system\nLibrary          SeleniumLibrary\n\n*** Variables ***\n${URL}                   https://example.com/easypay\n${INVALID_ORDER_DETAILS} invalid_order_details\n\n*** Test Cases ***\nValidate Customer Unable To Complete Payment For An Invalid Order\n    [Documentation] Test case for validating that the customer is unable to complete payment for an invalid order functionality of the EasyPay system\n    Launch EasyPay Application ${URL}\n    Enter Order Details ${INVALID_ORDER_DETAILS}\n    Attempt To Complete Payment\n    Validate Error Message Displayed\n'},
 'Description': {73: 'This test case checks the validation for being unable to

In [36]:
cleaned_df.to_json("generated_data.json")

In [40]:

# Concatenate DataFrames
concatenated_df = pd.concat([df, cleaned_df])

# Reset index if necessary
concatenated_df.reset_index(drop=True, inplace=True)

In [42]:
concatenated_df.head()

,Prompt,Response,Description,Testcase name
0,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for being...,Validate Unable To Create AutoPay For The Invo...
1,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for being...,Validate Unable To Create AutoPay For The Invo...
2,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for sched...,Validate Schedule AutoPay Payments On A Daily ...
3,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for sched...,Validate Schedule AutoPay For Credit Card
4,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for sched...,Validate Schedule AutoPay For Debit Card


In [43]:
concatenated_df.to_parquet(path="robot_test_framework_data.parquet")

In [50]:
# Remove duplicates based on column 'A'
df_no_duplicates = concatenated_df.drop_duplicates(subset='Prompt', keep='first')

# Reset index and drop the old index column
df_no_duplicates.reset_index(drop=True, inplace=True)

In [51]:
df_no_duplicates.to_parquet(path="robot_test_framework.parquet")

In [48]:
df_no_duplicates.shape

(1002, 4)

In [55]:
df_no_duplicates["instruction"] = "You are an expert assistant in Robot Framework. Based on user prompts, generate accurate and effective Robot Framework keyword test cases. Ensure your responses are clear and ready for implementation."

C:\Users\Area-51m\AppData\Local\Temp\ipykernel_19980\1301285521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates["instruction"] = "You are an expert assistant in Robot Framework. Based on user prompts, generate accurate and effective Robot Framework keyword test cases. Ensure your responses are clear and ready for implementation."


In [56]:
df_no_duplicates.sample(5)

,Prompt,Response,Description,Testcase name,instruction
991,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case validates the ability of a merc...,Validate Customer Unable To Process Payment Wi...,You are an expert assistant in Robot Framework...
506,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Test case...,This test case checks the validation for a use...,Validate User Is Able To Change Password,You are an expert assistant in Robot Framework...
125,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for viewi...,Validate View Saved Credit Card Details,You are an expert assistant in Robot Framework...
294,Generate a robot framework test case for testi...,*** Settings ***\r\nDocumentation Tes...,This test case checks the validation for selec...,Validate Select Pages From Dropdown In Specifi...,You are an expert assistant in Robot Framework...
687,Generate a robot framework test case for testi...,*** Settings ***\nDocumentation Test ...,This test case verifies that users are restric...,Validate Update Shipping Address Post Order Pl...,You are an expert assistant in Robot Framework...


In [59]:
list_of_dicts = df_no_duplicates[["Prompt", "Response", "instruction"]].to_dict(orient='records')

In [60]:
with open('dataset.json', 'w') as json_file:
    json.dump(list_of_dicts, json_file)